<a href="https://colab.research.google.com/github/pollyjuice74/ECCT/blob/main/ECCT_on_5G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/pollyjuice74/ECCT
import os
os.chdir('ECCT')



fatal: destination path 'ECCT' already exists and is not an empty directory.
